In [25]:
import requests
import pandas as pd

# Function to fetch data from an API
def fetch_api_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from {url}")
        return {}

# Assuming these are your API URLs and headers
url1 = "https://api.opensea.io/api/v2/collections/mutant-ape-yacht-club/stats"
url2 = "https://api.opensea.io/api/v2/collections/mutant-ape-yacht-club"
url3 = "https://api.opensea.io/api/v2/events/collection/mutant-ape-yacht-club"
headers1 = {"accept": "application/json", "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"}
headers2 = {"accept": "application/json", "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"}
headers3 = {"accept": "application/json", "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"}

# Fetch data from both APIs
data1 = fetch_api_data(url1, headers1)
data2 = fetch_api_data(url2, headers2)
data3 = fetch_api_data(url3, headers3)

In [26]:
# Process and convert data from the first API
df1_total = pd.DataFrame([data1['total']])
df1_intervals = pd.DataFrame(data1['intervals'])

In [27]:
# Process and convert data from the second API
collection_fields = ['collection', 'name', 'description', 'image_url', 'banner_image_url', 'owner', 'category', 'is_disabled', 'is_nsfw', 'trait_offers_enabled', 'collection_offers_enabled', 'opensea_url', 'project_url', 'wiki_url', 'discord_url', 'telegram_url', 'twitter_username', 'instagram_username', 'total_supply', 'created_date']
df2_collection = pd.DataFrame([{field: data2.get(field, '') for field in collection_fields}])
df2_contracts = pd.DataFrame(data2['contracts'])
df2_fees = pd.DataFrame(data2['fees'])
df2_payment_tokens = pd.DataFrame(data2['payment_tokens'])
df2_rarity = pd.DataFrame([data2['rarity']])

In [28]:
events_normalized = []
for event in data3['asset_events']:
    # Initialize event_data with event fields, excluding nft, asset, payment, criteria
    event_data = {k: v for k, v in event.items() if k not in ['nft', 'asset', 'payment', 'criteria']}
    
    # Check and add nft data if present
    if 'nft' in event and event['nft'] is not None:
        nft_data = {'nft_' + k: v for k, v in event['nft'].items()}
        event_data.update(nft_data)
    
    # Check and add asset data if present
    elif 'asset' in event and event['asset'] is not None:
        asset_data = {'asset_' + k: v for k, v in event['asset'].items()}
        event_data.update(asset_data)
    
    # Check and include payment data if necessary and present
    if 'payment' in event and event['payment'] is not None:
        payment_data = {'payment_' + k: v for k, v in event['payment'].items()}
        event_data.update(payment_data)
    
    events_normalized.append(event_data)

df_events = pd.DataFrame(events_normalized)

In [29]:
# Export to CSV
df1_total.to_csv('api1_total_stats.csv', index=False)
df1_intervals.to_csv('api1_intervals_stats.csv', index=False)
df2_collection.to_csv('api2_collection_info.csv', index=False)
df2_contracts.to_csv('api2_collection_contracts.csv', index=False)
df2_fees.to_csv('api2_collection_fees.csv', index=False)
df2_payment_tokens.to_csv('api2_collection_payment_tokens.csv', index=False)
df2_rarity.to_csv('api2_collection_rarity.csv', index=False)
df_events.to_csv('api3_events.csv', index=False)
